# Gaussian Copula Imputer
Welcome to this tutorial on the **Gaussian Copula Imputer**!
In this notebook, we demonstrate how to use the **Gaussian Copula Imputer** in Python.

This method improves the Gaussian Conditional Imputer by handling more realistic data.  
Instead of assuming that all features follow a normal distribution, it uses the actual distribution of each feature and models their dependencies with a Gaussian copula.

## Table of Contents
1. What is a Gaussian Copula Imputer?
2. Why do we need it?
3. How does the implementation work?
4. Example usage
5.  Summary


## 1. What is a Gaussian Copula Imputer?

- Goal: Realistically impute missing feature values
- Approach: Transform data into the Gaussian space using ECDFs (Empirical Cumulative Distribution Functions)
- Impute missing values conditinally asssuming a multivariate Gaussian in z-space
- Inverse transform to original feature space using inverse ECDFs

## 2. Why do we need it?
- Many classical methods (e.g., `shap`) assume feature independence
- → Leads to biased or unrealistic imputations
- Real-world data often contains dependencies between features
- The Gaussian Copula Imputer models those dependencies via the correlation structure in z-space
- → Result: more realistic imputations and more accurate Shapley values

## 3. How does the implementation work?
In this case section, we walk through the implementation of the `GaussianCopulaImputer` class step by step.
This class extends the `Imputer` interface from the `shapiq` framework and applies Gaussian Copula theory to impute missing values

### 3.1 Imports and Class Overview
First we import the required libraries and outline the overall structure of the `GaussianCopulaImputer` class

In [1]:
import numpy as np
from scipy.stats import multivariate_normal, norm
from numpy.linalg import pinv
from shapiq.games.imputer.base import Imputer

### 3.2 Constructor (`__init__`)
The constructor initializes the imputer with a model, input data, and an optional reference instance `x`.
It also prepares the structures for the ECDF transformations and the correlation matrix in Gaussian space.

In [3]:
class GaussianCopulaImputer(Imputer):
    def __init__(self, model=None, data=None, x=None):
        super().__init__(model=model, data=data)
        self._x_internal = x
        self.ecdfs = []
        self.inverse_ecdfs = []
        self.transformed_data = None
        self.correlation = None

### 3.3 Fitting the Imputer (`fit` method)
The `fit()` method transforms the data into Gaussian (z) space using ECDFs (Empirical Cumulative Distribution Functions). It stores:
- CDF and inverse ECDF for each feature
- Transformed data in z-space
- The correlation matrix in z-space

This step is essential before imputation can be done.

In [4]:
def fit(self, x: np.ndarray) -> None:
    self._x_internal = x
    x = np.asarray(x)
    n, d = x.shape
    self.ecdfs = []
    self.inverse_ecdfs = []
    z_data = np.zeros_like(x)

    for j in range(d):
        col = x[:, j]
        sorted_col = np.sort(col)

        def ecdf_func(x_val: float, sorted_col=sorted_col) -> float:
            return float(np.searchsorted(sorted_col, x_val, side="right") / len(sorted_col))

        def inverse_ecdf_func(p: float, sorted_col=sorted_col) -> float:
            p = np.clip(p, 1e-6, 1 - 1e-6)
            idx = np.round(p * (len(sorted_col) - 1)).astype(int)
            return sorted_col[idx]

        self.ecdfs.append(ecdf_func)
        self.inverse_ecdfs.append(inverse_ecdf_func)

        u = np.array([ecdf_func(xi) for xi in col])
        u = np.clip(u, 1e-6, 1 - 1e-6)
        z = norm.ppf(u)
        z_data[:, j] = z

    self.transformed_data = z_data
    with np.errstate(divide="ignore", invalid="ignore"):
        corr = np.corrcoef(z_data, rowvar=False)
        corr = np.nan_to_num(corr, nan=0.0, posinf=0.0, neginf=0.0)

    self.correlation = corr

### 3.4 Imputing Missing Values (`impute` method)
This method performs the actual imputation using the Gaussian copula.
Steps:
1. Transform known features into z-space using ECDFs.
2. Compute conditional mean and covariance in z-space.
3. Sample missing features from the conditional distribution.
4. Transform sampled values back to original space using inverse ECDFs.

This allows imputing missing features based on the known ones while preserving feature dependencies.

In [5]:
def impute(
    self, x_known: np.ndarray, known_idx: list[int], missing_idx: list[int]
) -> np.ndarray:
    z_known = np.array([
        norm.ppf(np.clip(self.ecdfs[idx](x_known[i]), 1e-6, 1 - 1e-6))
        for i, idx in enumerate(known_idx)
    ])

    assert self.correlation is not None, "Correlation matrix must be computed before imputation."

    Sigma = self.correlation
    Sigma_SS = Sigma[np.ix_(known_idx, known_idx)]
    Sigma_barS_S = Sigma[np.ix_(missing_idx, known_idx)]
    Sigma_barS_barS = Sigma[np.ix_(missing_idx, missing_idx)]
    Sigma_S_barS = Sigma_barS_S.T

    mu_S = np.zeros(len(known_idx))
    mu_barS = np.zeros(len(missing_idx))
    delta = z_known - mu_S
    inv_Sigma_SS = pinv(Sigma_SS)

    z_barS_cond_mean = mu_barS + Sigma_barS_S @ inv_Sigma_SS @ delta
    z_barS_cond_cov = Sigma_barS_barS - Sigma_barS_S @ inv_Sigma_SS @ Sigma_S_barS

    z_missing = multivariate_normal(
        mean=z_barS_cond_mean, cov=z_barS_cond_cov, allow_singular=True
    ).rvs()

    if len(missing_idx) == 1:
        z_missing = np.array([z_missing])

    x_missing = []
    for i, idx in enumerate(missing_idx):
        inv_ecdf = self.inverse_ecdfs[idx]
        percentile = norm.cdf(z_missing[i])
        x_val = inv_ecdf(percentile)
        x_missing.append(x_val)

    return np.array(x_missing)

### 3.5 Predicting with Coalitions (`__call__` method)
This method evaluates the model's output for different coalitions (feature subsets).
Each coalition is a binary vector indicating which features are known (1) and which are missing (0).
Steps:
1. Identify known and missing feature indices.
2. Impute missing features using the `impute` method.
3. Construct a full feature vector and predict the model output.

If no features are known, the model returns `0.0`.

In [6]:
def __call__(self, coalitions: np.ndarray) -> np.ndarray:
    values = []
    for coalition in coalitions:
        known_idx = list(np.where(coalition)[0])
        missing_idx = list(np.where(~coalition)[0])

        if len(known_idx) == 0:
            values.append(0.0)
            continue

        assert self.x is not None, "Reference instance x must be set before calling imputer."

        x_known = self.x[0, known_idx]
        x_imputed = self.impute(x_known, known_idx, missing_idx)

        full_x = np.zeros(self.x.shape[1])
        full_x[known_idx] = x_known
        full_x[missing_idx] = x_imputed

        values.append(float(self.predict(full_x.reshape(1, -1))[0]))

    return np.array(values)

## 4. Example Usage
We now demonstrate how to use the `GaussianCopulaImputer` with synthetic data and a simple dummy model.

### 4.1 Generate synthetic data and set up the imputer
We create a simple dataset of 100 samples and 5 features from a normal distribution, and define a dummy model that sums feature values. Then we initialize and fit the imputer.

In [ ]:
import numpy as np
#from shapiq_student.gaussian_copula_imputer import GaussianCopulaImputer
from gaussian_copula_imputer import GaussianCopulaImputer

class DummyModel:
    def __call__(self, X: np.ndarray) -> np.ndarray:
        return np.sum(X, axis=1)
    
    def predict(self, X: np.ndarray) -> np.ndarray:
        return np.sum(X, axis=1)

rng = np.random.default_rng(42)
data = rng.normal(loc=0, scale=1, size=(100, 5))
model = DummyModel()

imputer = GaussianCopulaImputer(model=model, data=data)
imputer.fit(data)

### 4.2 Impute missing values for a specific instance
We simulate a scenario where some features are missing and use the imputer to fill them in.

In [ ]:
x_sample = data[0, :]
known_idx = [0, 1, 3]
missing_idx = [2, 4]

x_known = x_sample[known_idx]
x_imputed = imputer.impute(x_known, known_idx, missing_idx)

print("Known values:", x_known)
print("Imputed values:", x_imputed)

### 4.3 Predict outcomes using feature coalitions
We provide coalitions (which features are known) and use the imputer to predict model outputs accordingly.

In [ ]:
x_ref = data[[0], :]
imputer._x_internal = x_ref

coalitions = rng.integers(0, 2, size=(5, 5))

predictions = imputer(coalitions)

print("Coalitions:\n", coalitions)
print("Predicted outputs:", predictions)

## 5. Summary
In this notebook, we explored the **Gaussian Copula Imputer**, a method for imputing missing feature values by:

- Transforming data into a Gaussian space using **ECDFs**
- Modeling dependencies via a **correlation matrix** in z-space
- Sampling missing values from a **conditional multivariate normal distribution**
- Transforming the imputed values back to the original feature space

Compared to methods that assume feature independence, this approach:
- Produces more realistic imputations
- Preserves dependencies between features
- Leads to more reliable results, especially in explainable AI settings (e.g., computing Shapley values)

The Gaussian Copula Imputer is a powerful tool for handling missing data in correlated datasets.